In [34]:
# This script performs the Bayesian Additive Regression Trees analysis on Todd Martin's data.
#
# Created: 01/19/2018 Wilson Melendez
# Revised: 

# Set location of main scripts.  The user has to adjust this pathname based on the location where this and 
# other scripts are located.
working_directory <- getwd()

# Set working directory
setwd(working_directory)

# Set pathname of location of Todd's data. 
toddFolder <- working_directory

In [35]:
# Set JAVA_HOME to the location of the JDK in your system.
# Sys.setenv("JAVA_HOME"="C:\\Program Files\\Java\\jdk1.8.0_152")

#  Get the location of the JDK
Sys.getenv("JAVA_HOME")

# Load the rJava package -- this is needed by the bartMachine.
library(rJava)

# Load the tictoc library
library(tictoc)

# Allocate memory needed before loading the bartMachine.
# Note that the maximum amount of memory can be set only once at the beginning of the R session (a
# limitation of rJava since only one Java Virtual Machine can be initiated per R session), but the number of
# cores can be respecified at any time.
options(java.parameters = "-Xmx16g")

# Load the bartMachine package.
library(bartMachine)

# Allocate number of cores that will be used by the bartMachine.
set_bart_machine_num_cores(4)

[1] "/usr/lib/jvm/java-8-openjdk-amd64/"

bartMachine now using 4 cores.


In [36]:
# Paste the names of the files to the directory pathanme.
filename1 <- paste(toddFolder, "/LC50_training_set-2d.csv", sep = "")
filename2 <- paste(toddFolder, "/LC50_prediction_set-2d.csv", sep = "")

# Read in CSV files.
Todd_TrainingData <- read.csv(filename1, stringsAsFactors=FALSE)
Todd_TestData <- read.csv(filename2, stringsAsFactors=FALSE)

In [37]:
# Extract the training X and Y matrices 
Xtraining <- Todd_TrainingData
Xtraining$CAS <- NULL
Xtraining$Tox <- NULL
Ytraning <- Todd_TrainingData$Tox

In [38]:
# Convert any non-numeric column of the X matrix to numeric.
numCols = ncol(Xtraining)   # Get the number of columns.
for (i in 1:numCols)
{
  if (class(Xtraining[,i]) != "numeric")
  {
    Xtraining[,i] <- as.numeric(Xtraining[,i])
  }
}

In [39]:
# Extract the X and Y test matrices.
Xtest <- Todd_TestData
Xtest$CAS <- NULL
Xtest$Tox <- NULL
Ytest <- Todd_TestData$Tox

In [40]:
# Convert any non-numeric column of the X test matrix to numeric.
numCols = ncol(Xtest)  # Get the number of columns
for (i in 1:numCols)
{
  if (class(Xtest[,i]) != "numeric")
  {
    Xtest[,i] <- as.numeric(Xtest[,i])
  }
}

In [41]:
tic.clearlog()
tic(msg = "BartMachine with Default Parameters")
# Build a BART model using default values.
bart_machine_todd <- bartMachine(Xtraining, Ytraning, 
                                 num_trees = 200,
                                 num_burn_in = 250,
                                 num_iterations_after_burn_in = 1000,
                                 alpha = 0.95, beta = 2, k = 2, q = 0.9, nu = 3,
                                 prob_rule_class = 0.5,
                                 mh_prob_steps = c(2.5, 2.5, 4)/9,
                                 debug_log = FALSE,
                                 run_in_sample = TRUE,
                                 s_sq_y = "mse",
                                 sig_sq_est = NULL,
                                 cov_prior_vec = NULL,
                                 use_missing_data = FALSE, 
                                 covariates_to_permute = NULL,
                                 num_rand_samps_in_library = 10000,
                                 use_missing_data_dummies_as_covars = FALSE,
                                 replace_missing_data_with_x_j_bar = FALSE,
                                 impute_missingness_with_rf_impute = FALSE,
                                 impute_missingness_with_x_j_bar_for_lm = TRUE,
                                 mem_cache_for_speed = TRUE,
                                 serialize = FALSE,
                                 seed = NULL,
                                 verbose = TRUE)
                            
toc(log = TRUE)
log.txt <- tic.log(format = TRUE)
print(unlist(log.txt))

bartMachine initializing with 200 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 798 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for regression ...
evaluating in sample data...done
BartMachine with Default Parameters: 12.588 sec elapsed
[1] "BartMachine with Default Parameters: 12.588 sec elapsed"


In [42]:
# Print a summary of the results.
summary(bart_machine_todd)

bartMachine v1.2.3 for regression

training data n = 659 and p = 797 
built in 10.5 secs on 4 cores, 200 trees, 250 burn-in and 1000 post. samples

sigsq est for y beforehand: 2.237 
avg sigsq estimate after burn-in: 0.37366 

in-sample statistics:
 L1 = 233 
 L2 = 151.22 
 rmse = 0.48 
 Pseudo-Rsq = 0.8973
p-val for shapiro-wilk test of normality of residuals: 0 
p-val for zero-mean noise: 0.95758 



In [43]:
# Make predictions on the training data. Note: this is not necessary in this case because the bart_machine_todd
# object has predicted values stored in it.  Consider this an example on how to use the "predict" function.
y_pred_training <- predict(bart_machine_todd, Xtraining)

# Save bartMachine object in a file.
saveRDS(bart_machine_todd, file = "bart_machine.todd.rds")

# Save predicted training values to a CSV file.
df_training <- data.frame(Ytraning, y_pred_training)
write.csv(df_training, "Predicted_Training_Values_JN.csv")

In [44]:
# Make predictions on the test data.  
y_pred_test <- predict(bart_machine_todd, Xtest)

# Write predicted values to a file.
saveRDS(y_pred_test, file = "PredictedValues_Todd_TestData.rds")

# Save predicted test values to a CSV file.
df_test <- data.frame(Ytest, y_pred_test)
write.csv(df_test, "Predicted_Test_Values_JN.csv")

# We can test model performance on out-of-sample test data for which the outcomes are known.  
oos_perf <- bart_predict_for_test_data(bart_machine_todd, Xtest, Ytest)
print(oos_perf$rmse)  # Print the root-mean-square error

# Calculate Q2 using the Java program's formula.
Q2 = 1.0 - sum((Ytest - y_pred_test)^2) / sum((Ytest - mean(Ytest))^2)
print(Q2)

[1] 0.7383556
[1] 0.7461869
